In [1]:
! pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 16.9 MB/s 
     |████████████████████████████████| 596 kB 54.5 MB/s 
     |████████████████████████████████| 101 kB 10.9 MB/s 
     |████████████████████████████████| 6.6 MB 55.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
from transformers import TFGPT2LMHeadModel, GPT2Tokenizer
gpt2tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

# add the EOS token as PAD token to avoid warnings
gpt2 = TFGPT2LMHeadModel.from_pretrained("gpt2", 
                                         pad_token_id=gpt2tokenizer.eos_token_id)

Downloading:   0%|          | 0.00/0.99M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/475M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at gpt2.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


In [3]:
# Generating text using a greedy search algorithm
# encode context the generation is conditioned on
input_ids = gpt2tokenizer.encode('Robotics is the ', return_tensors='tf')

# generate text until the output length (which includes the context length) reaches 50
greedy_output = gpt2.generate(input_ids, max_length=50)

print("Output:\n" + 50 * '-')
print(gpt2tokenizer.decode(greedy_output[0], skip_special_tokens=True))


Output:
--------------------------------------------------
Robotics is the vernacular of the future.

The future is not a future where robots are going to be able to do anything. It's a future where robots are going to be able to do anything.

The future is


A prompt was supplied for the model to complete. The model started in a promising manner but soon resorted to repeating the same output. The out for the text generation is indicative. You may see various outputs for the same prompt. There are a few various reasons for this. There is some inherent randomness that is built into this process, which we can try and control by setting random seeds. The models themselves may be retrained periodically by the Hugging face team and may evolve with the newer version. Beam search can be consicered as an alternative. At each step of generating a token, a set of top probability tokens are kept as part of the beam instead of just the highest- probability token. The sequence with the highest overall probability is returned at  the end of the generation. It is a time to generate text with beam search algorithm. 

In [4]:
import tensorflow as tf
tf.random.set_seed(42)  # for reproducible results
# BEAM SEARCH
# activate beam search and early_stopping
beam_output = gpt2.generate(
    input_ids, 
    max_length=51, 
    num_beams=20, 
    early_stopping=True
)

print("Output:\n" + 50 * '-')
print(gpt2tokenizer.decode(beam_output[0], skip_special_tokens=True))

Output:
--------------------------------------------------
Robotics is the vernacular of science fiction and fantasy. It's a genre that has been around for a long time. It's a genre that has been around for a long time. It's a genre that has been around for a long time


Qualitatively, the first sentence makes a lot more sense than the one generated by the greedy search. The early_stopping patameter signals generation to stop when all beams reach the EOS token. Howeverm there is still much repetition going on. One parameter that can be used to control the repetition is by setting a limit on n-grams being repeated.